<a href="https://colab.research.google.com/github/vikpy/mtechdatascience_mlcourse/blob/master/lec4_kfold_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import math
import random

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import*
from sklearn import datasets,linear_model

In [0]:
Auto=pd.read_csv("/content/drive/My Drive/data_ml_course/Auto.csv",header=0,na_values='?')
Auto=Auto.dropna().reset_index(drop=True)

In [4]:
Auto.head(5)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [5]:
Auto.shape

(392, 9)

In [0]:
np.random.seed(1)
train=np.random.choice(Auto.shape[0],196,replace=False)
select=np.in1d(range(Auto.shape[0]),train)

In [7]:
import statsmodels.formula.api as smf
lm=smf.ols('mpg~horsepower',data=Auto[select]).fit()
print(lm.summary())
precd=lm.predict(Auto)
square_error=(Auto['mpg']-precd)**2
print('----------Test error for 1st order---------')
print(np.mean(square_error[~select]))

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     316.4
Date:                Fri, 31 Jan 2020   Prob (F-statistic):           1.28e-42
Time:                        19:55:16   Log-Likelihood:                -592.07
No. Observations:                 196   AIC:                             1188.
Df Residuals:                     194   BIC:                             1195.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.3338      1.023     39.416      0.0

In [8]:
print('----------Test error for 2nd order---------')
lm2=smf.ols('mpg~horsepower+I(horsepower**2)',data=Auto[select]).fit()
print(lm2.summary())
precd=lm2.predict(Auto)
square_error=(Auto['mpg']-precd)**2

print(np.mean(square_error[~select]))

----------Test error for 2nd order---------
                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.720
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     248.0
Date:                Fri, 31 Jan 2020   Prob (F-statistic):           4.60e-54
Time:                        19:55:19   Log-Likelihood:                -562.15
No. Observations:                 196   AIC:                             1130.
Df Residuals:                     193   BIC:                             1140.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

In [9]:
print('----------Test error for 3nd order---------')
lm3=smf.ols('mpg~horsepower+I(horsepower**2)+I(horsepower**3)',data=Auto[select]).fit()
print(lm3.summary())
precd=lm3.predict(Auto)
square_error=(Auto['mpg']-precd)**2

print(np.mean(square_error[~select]))

----------Test error for 3nd order---------
                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     165.9
Date:                Fri, 31 Jan 2020   Prob (F-statistic):           4.60e-53
Time:                        19:55:29   Log-Likelihood:                -561.56
No. Observations:                 196   AIC:                             1131.
Df Residuals:                     192   BIC:                             1144.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

In [10]:
# Cross Validation
from sklearn.model_selection import KFold,cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

x=pd.DataFrame(Auto.horsepower)
y=Auto.mpg

model=LinearRegression()
model.fit(x,y)
print(model.intercept_)
print(model.coef_)

k_fold=KFold(n_splits=x.shape[0]) #loo use folds equal to of observations
test=cross_val_score(model,x,y,cv=k_fold,scoring='neg_mean_squared_error',n_jobs=-1)
print(test)
print(np.mean(-test))

A=[]
for porder in range(1,6):
    model=Pipeline([('poly',PolynomialFeatures(degree=porder)),('linear',LinearRegression())])
    k_fold=KFold(n_splits=x.shape[0])
    test=cross_val_score(model,x,y,cv=k_fold,scoring='neg_mean_squared_error',n_jobs=-1)
    print("error")
    A.append(np.mean(-test))
    print(A)
print(A)


B=[]
for pordera in range(1,7):
    model1=Pipeline([('poly',PolynomialFeatures(degree=pordera)),('linear',LinearRegression())])
    k_fold=KFold(n_splits=10)
    test1=cross_val_score(model1,x,y,cv=k_fold,scoring='neg_mean_squared_error',n_jobs=-1)
    print("error")
    B.append(np.mean(-test))
    print(B)
print(B)

39.93586102117047
[-0.15784473]
[-2.02001002e+00 -1.25092412e+00 -3.06805164e+00 -6.79901984e-02
 -7.08255629e-01 -4.13566745e+01 -8.13755358e+01 -6.71494767e+01
 -9.70498847e+01 -2.63430368e+01 -3.67428697e+00 -4.70741691e-01
 -1.60507789e+00 -9.70498847e+01 -8.89557151e-01 -8.69418110e+00
 -4.41228965e+01 -3.06562225e+01 -9.16549732e-01 -4.53185378e+01
 -1.45705281e+00 -3.00983554e+00 -3.54617605e-03 -1.52964088e+01
 -2.25022208e+01 -1.67905446e+01 -2.76735351e+00 -1.85354648e+01
 -2.29957474e-01 -9.16549732e-01 -5.18379999e+00 -3.54617605e-03
 -2.66745510e+01 -5.44791120e+01 -5.14078324e+01 -4.99405256e+01
 -3.80360006e+01 -1.19884585e-02 -2.91033003e+00 -3.23104363e+00
 -5.16691118e+00 -2.32487332e-01 -1.06678908e-02 -4.82615264e-01
 -2.10211114e+01 -4.35585197e+01 -2.66745510e+01 -6.51231162e+01
 -1.13690418e+01 -5.18379999e+00 -1.25085727e+00 -4.27873211e+00
 -1.77161819e+00 -3.58044876e+01 -1.21519855e+01 -8.41044041e+00
 -8.89557151e-01 -5.36657259e+00 -7.17595842e+01 -3.302308

In [12]:
portfolio=pd.read_csv("/content/drive/My Drive/data_ml_course/Portfolio.csv")
portfolio

,X,Y
0,-0.895251,-0.234924
1,-1.562454,-0.885176
2,-0.417090,0.271888
3,1.044356,-0.734198
4,-0.315568,0.841983
...,...,...
95,0.479091,1.454774
96,-0.535020,-0.399175
97,-0.773129,-0.957175
98,0.403634,1.396038


In [0]:
def alpha_fn(data,index):
    x=data.X[index]
    y=data.Y[index]
    return (np.var(y)-np.cov(x,y)[0,1])/(np.var(x)+np.var(y)-2*np.cov(x,y)[0,1])

In [25]:
a=np.arange(0,100)
alpha_fn(portfolio, a)

0.5766511516104118

In [26]:
np.sort(np.random.choice(range(0,100),size=100,replace=True))

array([ 1,  2,  3,  3,  4,  5,  7,  8,  8,  9,  9, 12, 13, 13, 14, 14, 16,
       19, 20, 23, 24, 25, 26, 26, 27, 27, 30, 31, 31, 31, 31, 32, 32, 33,
       34, 36, 38, 38, 38, 38, 39, 40, 40, 40, 40, 42, 42, 44, 45, 46, 46,
       47, 48, 48, 51, 51, 57, 58, 60, 60, 61, 61, 62, 64, 64, 67, 67, 68,
       69, 70, 71, 72, 72, 73, 73, 75, 76, 76, 76, 77, 77, 78, 79, 79, 81,
       81, 82, 84, 85, 85, 86, 87, 88, 90, 90, 93, 93, 96, 97, 99])

In [27]:
alpha_fn(portfolio,np.random.choice(range(0,100),size=100,replace=True))

0.6852151851352586

In [0]:
def boot_python(data,input_fun,iteration):
    n=portfolio.shape[0]
    idx=np.random.randint(0,n,(iteration,n))
    stat=np.zeros(iteration)
    for i in range(len(idx)):
        stat[i]=input_fun(data,idx[i])
    return {'Mean': np.mean(stat),'STD':np.std(stat)}

In [29]:
boot_python(portfolio,alpha_fn,1000)

{'Mean': 0.5800359461501615, 'STD': 0.09481338968889269}